### <p style="font-family:Garamond,serif">This notebook is for scraping tweets. </p> 

In [5]:
#Install the twitter scraping library
!pip install snscrape

In [2]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
from nlpia.data.loaders import get_data
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Imp

### <p style="font-family:Garamond,serif"> These code are from NLP course's notebook 4.1 Task. I'm using the sns scrape to scrape data from Twitter. </p>

In [3]:
import snscrape.modules.twitter as sntwitter

In [8]:
#Initialise dataframe
hashtag_az_df = pd.DataFrame(columns = ["text"])
hashtag_az_df.shape

(0, 2)

In [4]:
#How many to grab
maxTweets = 100000

###Comment out methods below as appropriate 

# ---EITHER--- Keyword search
# keyword = 'data'
# data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

# ---OR--- User search
#user = 'ASTRAZENECAUK'
#data = sntwitter.TwitterSearchScraper('from:ASTRAZENECAUK').get_items()

# ---OR--- Hastag search
hashtag = '#AstraZeneca'
data = sntwitter.TwitterHashtagScraper(hashtag + '-filter:links -filter:replies').get_items()

#Set the label for this group
hashtag_astrazeneca = []

#Collect in array and pair each tweet with a class label
for i, tweet in enumerate(data):
    if i >= maxTweets:
        break   
    hashtag_astrazeneca = hashtag_astrazeneca + [[tweet.content]]


In [7]:
print(len(hashtag_astrazeneca))
print(type(str(hashtag_astrazeneca[0]))) #make sure that the data type in the array are stings
str(hashtag_astrazeneca[1]) #Check the tweets in the array

100000
<class 'str'>


"['🚨 Atención 🚨\\n\\n¡Solo por hoy!\\n\\nPara todos los mayores de 18 años que hayan recibido su 2da dosis hace 4 meses podrán acercarse hoy a @BulevarCC @titanplazacc o Carrera a recibir su 3ra dosis de #AstraZeneca \\n\\nAprovecha la excepción y recibe tu dosis de refuerzo\\n\\nRT🐦']"

### <p style="font-family:Garamond,serif">I initially tried to use TextBlob to filter out English Tweets, however, the function has limitation on the amount of input data per-day, so I had to give up on this function.</p>  

In [208]:
#from textblob import TextBlob, Word, Blobber

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

In [21]:
!python -m spacy download en_core_web_sm

#intall spacy language detect to filer language
!pip install spacy-langdetect 

In [1]:
#create an array for storing English Tweets only
en_hashtag_astrazeneca = []

In [9]:
#import spacy for language detection
#code sourse : https://stackoverflow.com/questions/66712753/how-to-use-languagedetector-from-spacy-langdetect-package
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [11]:
#This bit of code only need to run once !
def get_lang_detector(nlp, name):
    return LanguageDetector()
nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [13]:
text = str(hashtag_astrazeneca[4])
doc = nlp(text)
print(doc._.language)
type(doc._.language)

#turn dict into array 
lang_doc = list(dict.items(doc._.language))
lang_array = np.array(lang_doc)

print(lang_array)

{'language': 'id', 'score': 0.9999967098958428}
[['language' 'id']
 ['score' '0.9999980749772664']]


In [14]:
for i in range(100000):
    text = str(hashtag_astrazeneca[i])
    doc = nlp(text)
    doc._.language
    lang_doc = list(dict.items(doc._.language))
    lang_array = np.array(lang_doc)
    #print(lang_array)
    if lang_array[0][1] == 'en':
        en_hashtag_astrazeneca.append(text)
#print(en_lang)  


In [15]:
en_hashtag_astrazeneca[50]

"['What would prevent #Pfizer,  #BioNTech, #J&amp;J and #AstraZeneca from changing up the formula of their VX to adjust its lethality and create specific types of variants?']"

In [16]:
en_hashtag_astrazeneca_df = pd.DataFrame(en_hashtag_astrazeneca, columns = ["text"])
en_hashtag_astrazeneca_df.shape


(23868, 1)

In [97]:
#>>>For cleaning dataframe if store wrong<<<
#en_hashtag_pfizer_df.drop(en_hashtag_pfizer_df.index, inplace=True)
#df_en_lang.drop(df_en_lang.loc[:80103].columns, axis = 1)

In [100]:
!pip install langdetect

     |████████████████████████████████| 981 kB 6.5 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=f3438c88683b2ad9e93ec1a9b22a6fdb99c43247b419a2a8258683be2e2cb08e
  Stored in directory: /Users/chenxifang/Library/Caches/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [17]:
en_hashtag_astrazeneca_df

,text
0,"[""Those who got their vaccines like me in the ..."
1,"['#Pfizer , Johnson&amp;Johnson , #Moderna , #..."
2,['Crytoverse and Metaverse face any risk on sh...
3,['What happened to the Astra Zeneca vaccine? #...
4,"['I am not anti vaccine or,as they insult a #C..."
...,...
23863,['This is a move that will come to bite these ...
23864,['🇩🇪#Germany &amp; 🇨🇵#France joining 13 other ...
23865,['#AstraZeneca #Bundesgesundheitsminister plea...
23866,['I would also like to hear from anyone that h...


In [25]:
#Write to file
file_name = "en_hashtag_astrazeneca"
en_hashtag_astrazeneca_df.to_csv("data/" + file_name + ".tsv")

In [26]:
#Read back in
en_hashtag_astrazeneca= pd.read_csv("data/" + file_name + ".tsv")